## Benchmark halo.

In [ ]:
from shared.preface import *
import commah

sim_dir = f'L025N752/DMONLY/SigmaConstant00/high_res_1Halo'

# Box parameters and arrays.
with open(f'{sim_dir}/box_parameters.yaml', 'r') as file:
    box_setup = yaml.safe_load(file)
DM_mass = box_setup['Content']['DM Mass [Msun]']*Msun
nums_snaps = np.load(f'{sim_dir}/nums_snaps.npy')
zeds_snaps = np.load(f'{sim_dir}/zeds_snaps.npy')

FCT_h = box_setup['Cosmology']['h']
FCT_H0 = FCT_h*100*km/s/Mpc
FCT_Omega_M = box_setup['Cosmology']['Omega_M']
FCT_Omega_L = box_setup['Cosmology']['Omega_L']

@nb.njit
def rho_crit(z):
    """Critical density of the universe as a function of redshift, assuming
    matter domination, only Omega_m and Omega_Lambda in Friedmann equation. See 
    notes for derivation.

    Args:
        z (array): redshift

    Returns:
        array: critical density at redshift z
    """    
    
    H_squared = FCT_H0**2 * (FCT_Omega_M*(1.+z)**3 + FCT_Omega_L) 
    rho_crit = 3.*H_squared / (8.*Pi*G)

    return np.float64(rho_crit)


def halo_sample_z(z, snap, Mvir_z0, DM_mass, out_dir):

    # Get the DM halo mass (and the number of DM particles for sample).
    commah_output = commah.run('WMAP9', zi=0, Mi=Mvir_z0, z=z)
    Mz = commah_output['Mz'][0,0]*Msun
    num_DM = math.floor(Mz / DM_mass)

    # Get the concentration of the halo.
    c_200 = commah_output['c'][0,0]

    # Calculate R_200 and R_s ("virial" radius and scale radius).
    R_200 = np.power(Mz / (200*rho_crit(z)*4/3*Pi), 1./3.)
    R_s = R_200 / c_200

    # Construct projection function.
    def Proj(r, r_s, norm):
        x = r/r_s
        return (np.log(1+x) - (x/(1+x)))/norm

    # Construct inverse function. Needs to be without numerical units.
    f_200 = np.log(1+c_200) - (c_200/(1+c_200))
    invf = inversefunc(Proj, args=(R_s/kpc, f_200))  

    # Sample uniformly between [0,1] and project to corresponding radius.
    sample = np.sort(np.random.uniform(size=num_DM))
    r_sample = invf(sample)

    # Sample for angles and convert to cartesian coordinates.
    phis = np.random.uniform(0, 2*Pi, num_DM)  # uniform [0,2pi)
    cos_thetas = 2.*np.random.uniform(0, 1, num_DM) - 1  # uniform [-1,1)

    # Convert to cartesian coordinates.
    x = r_sample*np.cos(phis)*np.sqrt(1. - cos_thetas**2)
    y = r_sample*np.sin(phis)*np.sqrt(1. - cos_thetas**2)
    z = r_sample*cos_thetas
    coords = np.column_stack((x,y,z))

    np.save(f'{out_dir}/benchmark_halo_snap_{snap}.npy', coords)


benchmark_outdir = f'L025N752/DMONLY/SigmaConstant00/benchmark_halo_files'
if not os.path.exists(benchmark_outdir):
    os.makedirs(benchmark_outdir)

with ProcessPoolExecutor(24) as ex:
    ex.map(
        halo_sample_z, zeds_snaps, nums_snaps,
        repeat(Mvir_MW/Msun), repeat(DM_mass), repeat(benchmark_outdir)
    )

In [ ]:
benchmark_DM = np.array([
    len(np.load(f'{benchmark_outdir}/benchmark_halo_snap_{num}.npy')) 
    for num in nums_snaps
])

print(np.log10(Mvir_MW/Msun), np.log10(DM_mass*benchmark_DM[-1]/Msun))

nums_proxy = np.arange(12, 36+1)
plt.plot(nums_proxy, benchmark_DM); plt.show()

## Run scripts.

In [3]:
!python simulation_numerical.py -d L025N752/DMONLY/SigmaConstant00/low_res_1Halo_plus_benchmark -st benchmark -mg 12.0 -mr 0.6 -hn 5

********Numerical Simulation: Mode=benchmark********
Halo batch params (Rvir,Mvir,cNFW):
[[253.5523526   12.24058042   6.9286026 ]
 [158.62979351  11.62953177   8.03988676]
 [137.07912107  11.43929142   9.41292697]
 [141.82849159  11.48366794   2.6088865 ]
 [306.35586267  12.48705608  11.48652354]]
***********************************
benchmark halo; snapshot 0012
^C
Process ForkProcess-24:
Process ForkProcess-26:
Process ForkProcess-28:
Process ForkProcess-29:
Process ForkProcess-33:
Process ForkProcess-12:
Process ForkProcess-27:
Process ForkProcess-41:
Process ForkProcess-25:
Process ForkProcess-23:
Process ForkProcess-21:
Process ForkProcess-5:
Process ForkProcess-8:
Process ForkProcess-31:
Process ForkProcess-19:
Process ForkProcess-10:
Process ForkProcess-16:
Process ForkProcess-14:
Process ForkProcess-3:
Process ForkProcess-17:
Process ForkProcess-30:
Process ForkProcess-34:
Process ForkProcess-13:
Process ForkProcess-2:
Traceback (most recent call last):
  File "/gpfs/home4/zimm

In [ ]:
!python simulation_analytical.py -d L025N752/DMONLY/SigmaConstant00/low_res_1Halo_plus_benchmark --MW_halo --no-VC_halo --no-AG_halo

In [ ]:
!python simulation_numerical.py -d L025N752/DMONLY/SigmaConstant00/high_res_1Halo -st single_halos -mg 12.0 -mr 0.6 -hn 1

## Load, transform and plot simulation outputs.

In [10]:
from shared.preface import *
from shared.shared_functions import *

class analyze_simulation_outputs(object):

    def __init__(
        self, sim_dir, objects, plot_list,
        specify_halos=None, ylims=None,
    ):

        # Required:
        self.sim_dir = sim_dir
        self.objects = objects
        self.plot_list = plot_list
        
        self.fig_dir = f'figures/{sim_dir}'
        if not os.path.exists(self.fig_dir):
            os.makedirs(self.fig_dir)

        # Optional:
        self.ylims = ylims
        self.specify_halos = specify_halos

        # Neccessary arrays.
        self.mrange = np.load(f'{self.sim_dir}/neutrino_massrange_eV.npy')*eV
        self.mpicks = np.array([0.01, 0.05, 0.1, 0.3])

        if 'NFW_halo' in objects:

            batch_paths = glob.glob(
                f'{sim_dir}/neutrino_vectors_numerical_benchmark_halo_batch*.npy'
            )
            
            self.vectors_benchmark = []
            for batch_path in batch_paths:
                self.vectors_benchmark.append(np.load(batch_path))
            self.vectors_benchmark = np.squeeze(
                np.array(self.vectors_benchmark)
            )
            self.vectors_benchmark = np.array(self.vectors_benchmark)

            self.etas_benchmark = np.load(
                f'{sim_dir}/number_densities_numerical_benchmark_halo.npy'
            )/N0


        if 'box_halos' in objects:
            
            self.etas_numerical = []
            self.vectors_numerical = []

            if self.specify_halos is None:
                halo_num = len(np.load(
                    glob.glob(f'{sim_dir}/halo*params.npy')[0]
                ))
            else:
                halo_num = self.specify_halos

            for halo in range(1, halo_num+1): 
                
                # Find all batch paths belonging to current halo.
                batch_paths = glob.glob(
                    f'{sim_dir}/neutrino_vectors_numerical_halo{halo}_batch*.npy'
                )

                # Concatenate all vector batches into one array.
                vectors_halo = []
                for batch_path in batch_paths:
                    vectors_halo.append(np.load(batch_path))
                vectors_halo = np.squeeze(np.array(vectors_halo))

                # Append vectors.
                self.vectors_numerical.append(vectors_halo)

                # Append overdensities.
                self.etas_numerical.append(
                    np.load(
                        f'{sim_dir}/number_densities_numerical_halo{halo}.npy'
                    )/N0
                )

            self.etas_numerical = np.array(self.etas_numerical)
            self.vectors_numerical = np.array(self.vectors_numerical)


        if 'analytical_halo' in objects:

            batch_paths = glob.glob(
                f'{sim_dir}/neutrino_vectors_analytical_batch*.npy'
            )
            
            self.vectors_analytical = []
            for batch_path in batch_paths:
                self.vectors_analytical.append(np.load(batch_path))
            self.vectors_analytical = np.squeeze(
                np.array(self.vectors_analytical)
            )
            self.vectors_analytical = np.array(self.vectors_analytical)

            self.etas_analytical = np.load(
                f'{sim_dir}/number_densities_analytical.npy'
            )/N0


    def make_plots(self, most_likely=True):

        if 'overdensity_band' in self.plot_list:

            ### ------------- ###
            ### Setup figure. ###
            ### ------------- ###
            fig, ax = plt.subplots(1,1)

            ax.set_xscale('log')
            ax.set_yscale('log')
            ax.set_title(f'Overdensity band')
            ax.set_xlabel(r'$m_{\nu}$ [meV]')
            ax.set_ylabel(r'$n_{\nu} / n_{\nu, 0}$')

            if self.ylims is None:
                ax.set_ylim(1e-3, 1e1)
            else:
                ax.set_ylim(self.ylims[0], self.ylims[1])
            
            plt.grid(True, which="both", ls="-")

            savefig_args = dict(
                bbox_inches='tight'
            )


            ### ------------- ###
            ### Plot objects. ###
            ### ------------- ###

            if 'NFW_halo' in objects:

                plt.plot(
                    self.mrange*1e3, self.etas_benchmark-1, 
                    color='green', label='(Benchmark) NFW Halo'
                )

            if 'box_halos' in objects:

                etas_median = np.median(
                    self.etas_numerical, axis=0)
                etas_perc2p5 = np.percentile(
                    self.etas_numerical, q=2.5, axis=0)
                etas_perc97p5 = np.percentile(
                    self.etas_numerical, q=97.5, axis=0)
                etas_perc16 = np.percentile(
                    self.etas_numerical, q=16, axis=0)
                etas_perc84 = np.percentile(
                    self.etas_numerical, q=84, axis=0)
                
                ax.plot(
                    self.mrange*1e3, (etas_median-1), color='blue', 
                    label='Box Halos: medians'
                )
                ax.fill_between(
                    self.mrange*1e3, (etas_perc2p5-1), (etas_perc97p5-1), 
                    color='blue', alpha=0.2, label='Box Halos: 2.5-97.5 % C.L.'
                )
                ax.fill_between(
                    self.mrange*1e3, (etas_perc16-1), (etas_perc84-1), 
                    color='blue', alpha=0.3, label='Box Halos: 16-84 % C.L.'
                )

            if 'analytical_halo' in objects:

                plt.plot(
                    self.mrange*1e3, self.etas_analytical-1, 
                    color='red', ls='solid', label='Analytical Halo'
                )

            plt.legend(loc='lower right')
            ax.yaxis.set_major_formatter(ticker.FuncFormatter(y_fmt))
            plt.savefig(f'{self.fig_dir}/overdensity_band.pdf', **savefig_args)
            plt.close()


        if 'overdensity_evolution' in self.plot_list:
            
            ### ------------- ###
            ### Setup figure. ###
            ### ------------- ###

            fig, ax = plt.subplots(1,1)
            ax.set_title('Overdensities (redshift) evolution')
            ax.set_xlabel('z')
            ax.set_ylabel(r'$n_{\nu} / n_{\nu, 0}$')

            if self.ylims is None:
                ax.set_ylim(1e-3, 1e1)
            else:
                ax.set_ylim(self.ylims[0], self.ylims[1])
            

            savefig_args = dict(
                bbox_inches='tight'
            )

            ### ------------- ###
            ### Plot objects. ###
            ### ------------- ###

            # Convert velocities to mementa.
            p_arr, _ = velocity_to_momentum(
                self.vectors_analytical[...,3:6], self.mpicks
            )

            # Overdensities for each redshift as if it was the last in the sim.
            inds = np.arange(p_arr.shape[-1])
            etas_zeds = np.array(
                [number_density(p_arr[...,0], p_arr[...,z]) for z in inds]
            ).T/N0

            z_int_steps = np.load(f'{sim_dir}/z_int_steps.npy')
            colors = ['blue', 'orange', 'green', 'red']
            for j, m in enumerate(self.mpicks):
                ax.semilogy(
                    z_int_steps, etas_zeds[j]-1, 
                    c=colors[j], label=f'{m:.3f} eV'
                )

            # Invert ordering of items in legend (looks better since the curves 
            # of higher masses are higher up in the plot).
            handles, labels = ax.get_legend_handles_labels()
            ax.legend(handles[::-1], labels[::-1], loc='lower right')

            ax.yaxis.set_major_formatter(ticker.FuncFormatter(y_fmt))
            plt.savefig(
                f'{self.fig_dir}/overdensity_evolution.pdf', **savefig_args
            )
            plt.close()


        if 'phase_space' in self.plot_list:
            
            # Load necessary box and sim info.
            with open(f'{self.sim_dir}/sim_parameters.yaml', 'r') as file:
                sim_setup = yaml.safe_load(file)

            p_num = sim_setup['momentum_num']
            p_start = sim_setup['momentum_start']
            p_stop = sim_setup['momentum_stop']
            phis = sim_setup['phis']
            thetas = sim_setup['thetas']

            # Convert velocities to mementa.
            p_arr, y_arr = velocity_to_momentum(
                self.vectors_analytical[...,3:6], self.mpicks
            )
            p0_arr, p1_arr, y0_arr = p_arr[...,0], p_arr[...,-1], y_arr[...,0]

            # Sort.
            ind = p0_arr.argsort(axis=-1)
            p1_sort = np.take_along_axis(p1_arr, ind, axis=-1)
            y0_sort = np.take_along_axis(y0_arr, ind, axis=-1)

            if most_likely:
                # Each velocity has a batch of neutrinos.
                # (min. of each to represent most (likely) clustered ones)
                m_len = (len(self.mpicks))
                p1_blocks = p1_sort.reshape((m_len, p_num, phis*thetas))
                p1_final = np.min(p1_blocks, axis=-1)
                y0_blocks = y0_sort.reshape((m_len, p_num, phis*thetas))
                y0_final = y0_blocks[...,0]
            else:
                p1_final = p1_sort
                y0_final = y0_sort

            # Fermi Dirac of the final momenta.
            FDvals = Fermi_Dirac(p1_final)

            fig, axs = plt.subplots(2,2, figsize=(12,12))
            fig.suptitle(
                'Phase-space distr. "today" compared to Fermi-Dirac' ,
                fontsize=18
            )

            savefig_args = dict(
                bbox_inches='tight'
            )

            for j, m_nu in enumerate(self.mpicks):

                k = j
                i = 0
                if j in (2,3):
                    i = 1
                    j -= 2

                # Simulation phase-space distr. of neutrinos today.
                axs[i,j].loglog(
                    y0_final[k], FDvals[k], label='PS today (from sim)', c='red', alpha=0.9
                )

                # Fermi-Dirac phase-space distr.
                pOG = np.geomspace(
                    p_start*T_CNB, p_stop*T_CNB, FDvals.shape[-1])
                FDvalsOG = Fermi_Dirac(pOG)
                yOG = pOG/T_CNB
                axs[i,j].loglog(yOG, FDvalsOG, label='PS Fermi-Dirac', c='blue', alpha=0.7)

                # Escape momentum.
                '''
                Rvir_halo = halo_param_arr[0]*kpc
                Mvir_halo = 10**(halo_param_arr[1])*Msun
                cNFW_halo = halo_param_arr[2]
                rho0_halo = scale_density_NFW(0., cNFW_halo)*(Msun/kpc**3)
                rs_halo = Rvir_halo/cNFW_halo
                _, y_esc = escape_momentum(
                    FCT_init_xys, 0., 
                    rho0_halo, Mvir_halo, Rvir_halo, rs_halo, m_nu, 'none'
                )
                axs[i,j].axvline(y_esc, c='k', ls='-.', label='y_esc')
                '''

                # Plot styling.
                axs[i,j].set_title(f'{m_nu} eV')
                axs[i,j].set_ylabel('FD(p)')
                axs[i,j].set_xlabel(r'$y = p / T_{\nu,0}$')
                axs[i,j].legend(loc='lower left')
                axs[i,j].set_ylim(1e-5, 1e0)
                axs[i,j].set_xlim(p_start, 1e2)

            plt.savefig(
                f'{self.fig_dir}/phase_space.pdf', **savefig_args
            )
            plt.close()


def escape_momentum(x_i, z, ):
    ...


sim_dir = f'L025N752/DMONLY/SigmaConstant00/low_res_1Halo_plus_benchmark'
objects = (
    'NFW_halo', 
    'box_halos', 
    'analytical_halo'
)

Analysis = analyze_simulation_outputs(
    sim_dir = sim_dir,
    objects = objects,
    plot_list = (
        'overdensity_band', 
        'overdensity_evolution',
        # 'phase_space',
    ),
)

Analysis.make_plots(most_likely=False)


#? TODO LIST:
#? - finish escape momentum function and plot over analytical phase-space plot
#? - include option for numerical (box_halos and NFW_halo) for all plots
#? - 

In [ ]:
def transform_simulation_outputs_OLD(out_dir, sim_type):
    """
    sim_type specifies simulation mode: 
        'single_halos', 'all_sky', 'spheres', 'benchmark'.
    """

    # Try loading the equivalent output for the analytical simulation method.
    # If it hasn't been run yet, raise error and inform user.
    try:

        # Load neutrino vectors (positions and velocities) of the 10k batches, and concatenate them into a single array.
        batch_paths = glob.glob(
            f'{out_dir}/neutrino_vectors_analytical_batch*.npy'
        )
        vectors_ana = []
        for batch_path in batch_paths:
            vectors_ana.append(np.load(batch_path))
        vectors_ana = np.squeeze(np.array(vectors_ana))

        etas_ana = np.load(
            f'{out_dir}/number_densities_analytical.npy'
        )/N0

        analytical_out = True

    except FileNotFoundError:
        print('! Analytical simulation output not found !')
        analytical_out = False


    if 'single_halos' or 'spheres' in sim_type:
        
        # Load neutrino vectors (positions and velocities) of the 10k batches, and concatenate them into a single array.
        batch_paths = glob.glob(
            f'{out_dir}/neutrino_vectors_numerical_halo1_batch*.npy'
        )
        vectors_num = []
        for batch_path in batch_paths:
            vectors_num.append(np.load(batch_path))
        vectors_num = np.squeeze(np.array(vectors_num))

        etas_num = np.load(
            f'{out_dir}/number_densities_numerical_halo1.npy'
        )/N0

        if analytical_out:
            return vectors_ana, etas_ana, vectors_num, etas_num
        else:
            return vectors_num, etas_num
    
    else:
        # Load angle pairs and calculate overdensities for the all_sky mode.
        all_sky_output = np.load(f'{out_dir}/number_densities_numerical.npy')
        angle_pairs = all_sky_output[:, :2]
        etas_numerical = all_sky_output[:, 2:]/N0

        return etas_numerical, angle_pairs


def analyze_simulation_outputs_OLD(
        etas, m_nu_range, out_dir,
        plots_to_make, ylims, Mertsch=False,
        # vectors, sim_fullname,
    ):

    # Figure directory.
    fig_dir = f'figures/{out_dir}'
    if not os.path.exists(fig_dir):
        os.makedirs(fig_dir)

    savefig_args = dict(
        bbox_inches='tight'
    )

    if Mertsch:
        etas_analytical = etas[0,:]
        etas_numerical = etas[1:,:]
    else:
        etas_numerical = etas[...]

    ### ================================== ###
    ### Figure of merit: overdensity band. ###
    ### ================================== ###

    fig, ax = plt.subplots(1,1)

    if 'overdensity_band' in plots_to_make:

        if Mertsch:
            # Plot smooth simulation.
            ax.plot(
                m_nu_range*1e3, (etas_analytical-1), color='red', ls='solid', 
                label='Analytical simulation'
            )

        # '''
        if etas_numerical.ndim <= 1:
            ax.plot(
                m_nu_range*1e3, (etas_numerical-1), color='blue', 
                label='medians'
            )
        else:
            nus_median = np.median(etas_numerical, axis=0)
            nus_perc2p5 = np.percentile(etas_numerical, q=2.5, axis=0)
            nus_perc97p5 = np.percentile(etas_numerical, q=97.5, axis=0)
            nus_perc16 = np.percentile(etas_numerical, q=16, axis=0)
            nus_perc84 = np.percentile(etas_numerical, q=84, axis=0)
            ax.plot(
                m_nu_range*1e3, (nus_median-1), color='blue', 
                label='Halo sample medians'
            )
            ax.fill_between(
                m_nu_range*1e3, (nus_perc2p5-1), (nus_perc97p5-1), 
                color='blue', alpha=0.2, label='2.5-97.5 %'
            )
            ax.fill_between(
                m_nu_range*1e3, (nus_perc16-1), (nus_perc84-1), 
                color='blue', alpha=0.3, label='16-84 %'
            )
        # '''

        if Mertsch:
            # Plot endpoint values from Mertsch et al.
            x_ends = [1e1, 3*1e2]
            y_ends = [3*1e-3, 4]
            ax.scatter(x_ends, y_ends, marker='x', s=15, color='orange')

        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_title(f'Overdensity band')
        ax.set_xlabel(r'$m_{\nu}$ [meV]')
        ax.set_ylabel(r'$n_{\nu} / n_{\nu, 0}$')
        # ax.set_ylim(ylims[0], ylims[1])
        plt.grid(True, which="both", ls="-")
        plt.legend(loc='lower right')


        plt.savefig(f'{fig_dir}/overdensity_band.pdf', **savefig_args)
        plt.show()


box_name = 'L025N752'
box_ver = 'DMONLY/SigmaConstant00'
sim_info = 'low_res_1Halo_plus_benchmark'
sim_type = 'single_halos'
out_dir = f'{box_name}/{box_ver}/{sim_info}'
vecs_ana, etas_ana, vecs_num, etas_num = transform_simulation_outputs(
    out_dir, sim_type
)
etas_combined = np.stack((etas_ana, etas_num), axis=0)

neutrino_massrange = np.load(f'{out_dir}/neutrino_massrange_eV.npy')*eV

analyze_simulation_outputs(
    etas=etas_combined, 
    m_nu_range=neutrino_massrange,
    out_dir=out_dir,
    plots_to_make=('overdensity_band', 'vels'), 
    ylims=(1e-3, 1e2),
    # ylims=(0, 1e-2),
    Mertsch=True
)